In [1]:
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Model, GPT2Config, DataCollatorForLanguageModeling,BertTokenizer,BertModel,BertConfig
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import evaluate
evaluate.load
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [2]:
%pwd

'/home/ubuntu/ssk/LiGOFwlr/tutorial'

In [3]:
dataset = load_dataset("/home/ubuntu/ssk/openwebtext")

/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:

dataset["eval"] = dataset["train"].select(range(20000,20100))
dataset["train"] = dataset["train"].select(range(10000))

In [5]:
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(
        "/home/ubuntu/ssk/LiGOFwlr/tutorial/GPT2-tokenizer"
    )

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=128)
# def tokenize_function(examples):
#     return tokenizer.batch_encode_plus(
#         examples["text"], padding="max_length", truncation=True, max_length=1024, return_tensors="pt"
#     )

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=50)

In [6]:
tokenized_datasets["train"].set_format(type="torch", columns=["input_ids", "attention_mask"])

tokenized_datasets["eval"].set_format(type="torch", columns=["input_ids", "attention_mask"])

In [7]:
tokenized_datasets["train"][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
tokenized_datasets["train"][0]['attention_mask']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])

In [9]:
def shift_labels_right(inputs):
    labels = inputs["input_ids"].clone()
    labels[-1] = -100 # ignore_index
    labels[0:-1] = inputs["input_ids"][1:].clone()
    inputs["labels"] = labels
    return inputs
final_dataset = tokenized_datasets.map(shift_labels_right)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
final_dataset["train"][0]

{'input_ids': tensor([50256, 13924,    12,   559,    12, 35784,    11, 25051,   357, 18474,
             8,  1377, 45591,  4970,    11,  1319, 44556,   287,  2356,   290,
         44787,   379,  1204,    11,  7342,  7519,   290, 20669,  2513,  1497,
           422,   257,  2214,  4436,  3217,  1755,   706,   257, 21402,  3315,
          1074, 23724,   262,  1989,    11,  2282,   340,   373,  5213,   546,
          2324,    13,   198,   198,   464,  2551,  1364,  8100,  5953,  8366,
         34428,   298,  2986, 33708, 42095,   355,   262,   691,  6253,   379,
           262,  4436,   284,   651,   262,  3871,   832,   262,  1755,    13,
           198,   198, 18474,  7317,  2098,    11,  1912,   319, 10275,   351,
           617,   286,   262,  7519,    11,   326,   262,  1578,  7973,  6149,
           262, 21402,  3274, 22225,   290,  7929,  4816,   284, 36316,    13,
          2102,    11, 21402,  5953, 36831,  2269,   861,   402,  2926,    82,
            11,   257,  6253,   508,   

In [11]:
config_gpt2 = GPT2Config(n_positions=128,
                    n_embd=128,
                    n_layer=6,
                    n_head=8)
model= GPT2LMHeadModel(config_gpt2)

In [12]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(128, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=50257, bias=False)
)

In [13]:
training_args = TrainingArguments(
        output_dir="./logs",
        overwrite_output_dir=True,
        num_train_epochs=50,
        per_device_train_batch_size=16,
        save_steps=10_000,
        save_total_limit=2,
        remove_unused_columns=False,
        report_to="none",
        use_cpu=False,
        #evaluation_strategy="epoch"
    )

In [14]:
#data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
trainer = Trainer(
        model=model,
        args=training_args,
        #data_collator=data_collator,
        train_dataset=final_dataset ["train"],
        #eval_dataset=final_dataset ["eval"],
        #compute_metrics=compute_metrics,
        
    )
trainer.train()

/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,9.151600
1000,7.726600
1500,7.518500
2000,7.432800
2500,7.351600
3000,7.274200
3500,7.208100
4000,7.155100
4500,7.103300
5000,7.066700


/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=15650, training_loss=7.037701642216204, metrics={'train_runtime': 1288.0068, 'train_samples_per_second': 388.197, 'train_steps_per_second': 12.151, 'total_flos': 456916992000000.0, 'train_loss': 7.037701642216204, 'epoch': 50.0})

In [28]:
import torch

In [33]:
encoded_inputs = tokenizer.encode(["Good"])
encoded_inputs_tensor = torch.LongTensor(encoded_inputs).to("cuda")

In [34]:
generated_results = model.generate(encoded_inputs_tensor.unsqueeze(0))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [35]:
tokenizer.decode(generated_results.squeeze().cpu().numpy())

'<|endoftext|>Good is the of.�s the of,�s of,,,�s,'

In [37]:
import torch
import torchvision

In [ ]:
torchvision.datasets.flowers102